In [18]:
import os, sys

module_path = os.path.abspath(os.path.join('..')) # append directory one level up to import path
if module_path not in sys.path:  # ... if it hasn't been appended already
    sys.path.append(module_path)

import dsp_fpga_lib as dsp # import and set plot properties
import dsp_fpga_fix_lib as fx

import numpy as np
import numpy.random as rnd
from numpy import (pi, log10, exp, sqrt, sin, cos, tan, angle, arange,
                    linspace, array, zeros, ones)
from numpy.fft import fft, ifft, fftshift, ifftshift, fftfreq
# %matplotlib widget gibt interaktive plots, funktioniert aber nicht immer
%matplotlib widget
# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
#print(plt.style.available)
#plt.style.use('seaborn-poster') # sucht in stylelib Directories
#plt.style.use('../presentation.mplstyle') # sucht im Pfad
figsize = {"figsize":(12,8)}

from IPython.core.display import Image
from IPython.core.display import display_html

<!-- example for static HTML image embedded in MarkDown
![Menschen und Python](http://imgs.xkcd.com/comics/reassuring.png) 
MarkDown allows no alignment and scaling, but HTML/CSS does:

<p style="text-align:center; font-weight:bold">
<img src="http://imgs.xkcd.com/comics/reassuring.png" />
<br />Python macht glücklich ...
</p>
 -->


# Spektrale Fehlerformung
Wie in den vorigen Kapiteln gezeigt wurde, kann man den nicht-linearen Prozess der (Re-)Quantisierung beschreiben durch Addition des Quantisierungsfehlers $\epsilon(t)$ bzw. $\epsilon[n]$ (für zeitdiskrete Signale) zum Signal. Wenn der Quantisierungsfehler bestimmte Bedingungen erfüllt (stationärer Prozess mit unkorrelierten Werten), ist eine einfache lineare Beschreibung im Frequenzbereich möglich, da das Spektrum von $\epsilon[n]$ dann weiß ist.

<img src="./Linearized_Quantizer_2.png" width="600">

## Zeitdiskrete Systeme
Aufgrund der Multiplikationen mit den Koeffizienten nimmt die Wortlänge in digitalen Filtern immer zu. Bei rekursiven Filtern muss daher die Wortlänge reduziert (= requantisiert) werden bevor die Werte des rekursiven Pfads mit den Eingangswerten kombiniert werden.

In [19]:
NFFT = 2048; x = np.linspace(0,6*pi,NFFT);  a = 0.5*sin(2*x)
#q_obj = (1,2,'round','sat') # try 'floor' ; 'wrap'
q_obj = {'WI':0, 'WF': 5,                 # number of ingeger and fractional bits
          'quant':'floor', 'ovfl': 'wrap'} # quantization and overflow behaviour
Q = fx.Fixed(q_obj)                      # instance of fixpoint class Fixed()

aq = Q.fixp(a)
fig1=plt.figure(figsize=(FigSize[0]*2,FigSize[1])); fig1.clf()
ax1 = fig1.add_subplot(2,2,1); ax1.plot(x,aq)
print('Number of overflows = ', Q.N_over) 
ax1.plot(x, a, 'r', label = r'$a(x)$'); ax1.grid(True)
ax1.step(x, aq, where = 'post', label = r'$a_Q(x)$')
ax1.plot(x, a-aq, label = r'$a(x) - a_Q(x)$')
ax1.legend(fontsize = 14); ax1.set_xlabel(r'$x \rightarrow$')
ax2 = fig1.add_subplot(2,2,2)
A = np.fft.fft(a)/NFFT; f = np.fft.fftfreq(NFFT, 1e-3)
AQ = np.fft.fft(aq)/NFFT
ax2.plot(f[0:NFFT//2 - 1],20*log10(2*abs(A[0:NFFT//2-1])), label = r'$A(f)$'); ax2.grid(True)
ax2.plot(f[0:NFFT//2 - 1],20*log10(2*abs(AQ[0:NFFT//2-1])), label = r'$A_Q(f)$')
ax2.legend(fontsize = 14); ax2.set_xlabel(r'$f \rightarrow$')
ax2.set_ylim(-100,10);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Number of overflows =  0


## Sigma-Delta ADCs
$\Sigma \Delta$ ADCs enthalten sehr grobe Quantisierer (oft nur 1 Bit!), erzielen aber durch Noise Shaping *effective number of bits* (ENOB) von bis zu 24 Bits. In diesem Kapitel wird gezeigt, dass bei Systemen mit Überabtastung der Quantisierer verbessert werden kann, indem man Rauschen aus dem Nutzband zu höheren Frequenzen schiebt ("*noise shaping*" oder "*error spectral shaping*"). Dies wird erreicht, indem man das quantisierte Signal vom Eingangssignal subtrahiert ("Delta") und den Quantisierungsfehler aufsummiert ("Sigma"). Der Fehler wird abgetastet und grob quantisiert (im Bild mit einem Bit). Eine "1" dass der Fehler Das folgende Bild zeigt das Grundprinzip:

<p style="text-align:center; font-weight:bold">
<img src="adc_sigma_delta_a.png" align='center' width='70%'/>
<br />CT-Sigma-Delta ADC erster Ordnung
</p>

Indem man den Integrator über der Summierer "schiebt", erhält man eine einfachere Struktur, allerdings muss der Addierer jetzt eine höhere Bandbreite haben. Außerdem wurden Sampler und Quantizer vertauscht, da so der Quantizer eine ganze Periode Zeit für seine "Entscheidung" hat:

<p style="text-align:center; font-weight:bold">
<img src="adc_sigma_delta_b.png" align='center' width='70%'/>
<br />CT-Sigma-Delta ADC erster Ordnung
</p>

Modulatoren dieser Art kann man auch mit höherer Ordnung bauen wie im folgenden Bild gezeigt:

<p style="text-align:center; font-weight:bold">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/DeltaSigma2.svg/500px-DeltaSigma2.svg.png" align='center' width='70%'/>
<br />CT-Sigma-Delta ADC erster Ordnung
</p>


<p style="text-align:center; font-weight:bold">
<img src="http://upload.wikimedia.org/wikipedia/commons/f/f9/Fig._1b.svg" />
<br />Sigma-Delta Modulator erster Ordnung [wikipedia.org]
</p>